# 🔁 LSTM Modeling for Sequential Usage Patterns

In this notebook, we:
- Load sequential user behavior data
- Reshape into [samples, timesteps, features]
- Build and train an LSTM classifier
- Evaluate performance

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
df = pd.read_csv('../data/raw/lstm_usage_logs.csv')

# Pivot to shape (n_customers, timesteps, features)
features = ['usage_count', 'interaction_rate', 'avg_duration']
pivoted = df.pivot(index='customer_id', columns='day', values=features)

# Reshape for LSTM input
X = pivoted.values.reshape((len(pivoted), 30, len(features)))
y = df.drop_duplicates('customer_id').set_index('customer_id').loc[pivoted.index, 'churn'].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.25, random_state=42)

In [ ]:
model = Sequential([
    LSTM(64, input_shape=(30, 3), return_sequences=False),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['AUC'])
early_stop = EarlyStopping(monitor='val_loss', patience=3)

history = model.fit(X_train, y_train, epochs=20, batch_size=32,
                    validation_split=0.2, callbacks=[early_stop], verbose=1)

In [ ]:
y_prob_lstm = model.predict(X_test).flatten()
y_pred_lstm = (y_prob_lstm >= 0.5).astype(int)

print(classification_report(y_test, y_pred_lstm))
print('ROC AUC:', roc_auc_score(y_test, y_prob_lstm))

In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('LSTM Training Curve')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()